In [1]:
import numpy as np
from tqdm import trange


In [23]:
# load the mnist dataset

def fetch(url):
  import requests, gzip, os, hashlib, numpy
  fp = os.path.join("/tmp", hashlib.md5(url.encode('utf-8')).hexdigest())
  if os.path.isfile(fp):
    with open(fp, "rb") as f:
      dat = f.read()
  else:
    with open(fp, "wb") as f:
      dat = requests.get(url).content
      f.write(dat)
  return numpy.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()

# Shape(m, pixel rows, pixel cols) == (6000, 28, 28)
X_train = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
y_train = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]

# Normalizing pixel values from [0,1]
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [24]:
def layer_init(L_IN, L_OUT):
  # uniform dist
  ret = np.random.uniform(-1., 1., size=(L_OUT,L_IN))/np.sqrt(L_IN*L_OUT)
  return ret.astype(np.float32)


In [48]:
def propogate(X,y):
    '''handles forward and backwards propogation
    theta1 size: (hidden_layer_size , input_layer_size)
    theta2 size: (num_labels , hidden_layer_size)
    '''
    out = np.zeros((len(y),10), np.float32)
    out[range(out.shape[0]),y] = 1
    print(out)
    
    

In [51]:
# Architecture
input_layer_size = 784   # (28,28) pixels
hidden_layer_size = 120  # Hidden layer nodes
num_labels = 10          # digits [0-9] = 10 total

np.random.seed(1420)
theta1 = layer_init(input_layer_size, hidden_layer_size)  # (hidden_layer_size , input_layer_size)
theta2 = layer_init(hidden_layer_size, num_labels)        # (num_labels , hidden_layer_size)

learning_rate = 0.0001
batch_size = 12
epochs = 20

costs, accuracies = [], []

In [52]:


# Training logic
for i in (t := trange(epochs)):
    
    # Grabbing a random array of size batch size, indexes [0-m], m := # examples
    samples = np.random.randint(0, len(y_train), size=(batch_size))
    X = X_train[samples].reshape(-1, 28 * 28) # (m, pixel rows, pixel cols) -> (m, pixels)
    y = y_train[samples]                      # (m, ) vector of output MIGHT {NEED TO RESHAPE}
    
    propogate(X,y)
    
    pass




100%|██████████| 20/20 [00:00<00:00, 5199.01it/s]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
